### Задание 1

- F1@k
- MAR@K
- Coverage
- Intra-list similarity

### Задание 2

In [1]:
import numpy as np
import pandas as pd

#### hit_rate_at_k

In [66]:
recommended_list = np.array([144,156, 1134, 32, 1543, 3345, 533, 11, 143]) #id товаров
recommended_prices = np.array([144,156, 1134, 2233, 1543, 3345, 533, 11, 534])
bought_list = np.array([521, 32, 143, 991])
bought_prices = np.array([23,2233,534,56])

In [37]:
def hit_rate(recommended_list, bought_list):
    flags = np.isin(bought_list, recommended_list)
    return (flags.sum() > 0) * 1

In [38]:
hit_rate(recommended_list, bought_list)

1

In [39]:
def hit_rate_at_k(recommended_list, bought_list, k=5):
    return hit_rate(recommended_list[:k], bought_list)

In [40]:
hit_rate_at_k(recommended_list, bought_list, k=3)

0

In [41]:
hit_rate_at_k(recommended_list, bought_list, k=5)

1

#### money_recall_at_k

In [80]:
def recall(recommended_list, bought_list):
    flags = np.isin(bought_list, recommended_list)
    return flags.sum() / len(bought_list)
    

def recall_at_k(recommended_list, bought_list, k=5):
    return recall(recommended_list[:k], bought_list)

def money_recall(recommended_list, bought_list, prices_recommended, prices_bought):
    flags = np.isin(recommended_list, bought_list)
    return np.dot(flags, prices_recommended).sum() / prices_bought.sum()

def money_recall_at_k(recommended_list, bought_list, prices_recommended, prices_bought, k=5):
    return money_recall(recommended_list[:k], bought_list,  prices_recommended[:k], prices_bought)

In [81]:
recall(recommended_list, bought_list)

0.5

In [82]:
recall_at_k(recommended_list,bought_list,k=5)

0.25

In [83]:
money_recall_at_k(recommended_list,bought_list, recommended_prices, bought_prices, k=5)

0.7846099789177794

#### MAP@k

In [108]:
recommended_list_3_users = [[143, 156, 1134, 991, 27, 1543, 3345, 533, 11, 43], 
                    [1134, 533, 14, 4, 15, 1543, 1, 99, 27, 3345],
                    [991, 3345, 27, 533, 43, 143, 1543, 156, 1134, 1]]

bought_list_3_users = [[521, 32, 143],  # юзер 1
                       [143, 156, 991, 43, 11], # юзер 2
                       [1,2]] # юзер 3

In [91]:
def precision(recommended_list, bought_list):
    flags = np.isin(bought_list, recommended_list)
    return flags.sum() / len(recommended_list)

In [92]:
def precision_at_k(recommended_list, bought_list, k=5):
    return precision(recommended_list[:k], bought_list)

In [93]:
def ap_k(recommended_list, bought_list, k=5):
    relevant_indexes = np.nonzero(np.isin(recommended_list, bought_list))[0]
    if len(relevant_indexes) == 0:
        return 0
    
    amount_relevant = len(relevant_indexes)
    
    sum_ = sum([precision_at_k(recommended_list, bought_list, k=index_relevant+1) for index_relevant in relevant_indexes])
    return sum_/amount_relevant

In [94]:
ap_k(recommended_list, bought_list, k=5)

0.2361111111111111

In [98]:
def map_k(recommended_lists, bought_lists, k=5):
    n = len(recommended_lists)
    s = 0.0
    for i in range(n):
        s += ap_k(recommended_lists[i], bought_lists[i], k=5)
    return s / n

In [99]:
map_k(recommended_list_3_users, bought_list_3_users, k=5)

0.3333333333333333

#### NDCG@k

In [143]:
def discount(i):
    if(i <= 2):
        return i + 1
    return np.log2(i + 1)

def ndcg(recommended_list, bought_list):
    flags = np.isin(recommended_list, bought_list)
    n = len(flags)
    dcg = sum([float(flags[i]) / discount(i) for i in range(n)]) / n
    idcg = sum([1.0 / discount(i) for i in range(n)]) / n
    return dcg / idcg
    
def ndcg_at_k(recommended_list, bought_list, k=5):
    return ndcg(recommended_list[:k], bought_list)

In [144]:
ndcg_at_k(recommended_list, bought_list, k=5)

0.18089660299497992

#### MRR@k

In [113]:
def rank(recommended, first_relevant):
    try:
        return list(recommended).index(first_relevant)
    except ValueError:
        return 0

In [111]:
def reciprocal_rank(recommended_lists, bought_lists, k=1):
    n = len(recommended_lists)
    s = 0.0
    for i in range(n):
        r = rank(recommended_lists[i][:k], bought_lists[i][0])
        if r != 0:
            s += 1 / r
    return s / n
    pass

In [145]:
reciprocal_rank(recommended_list_3_users, bought_list_3_users, k=10)

0.037037037037037035